In [5]:
import pandas as pd
import json, datetime, os, glob, csv

#--------Variables to Change-------------
#Mininum number of seconds the song was played to be counted
minPlayTime = 30
#Folder where the spotify json files are located
pathToJson = '/content/json'
#----------------------------------------


#Name and details of the output csv file
outputFile = open('output.csv', mode = 'w' , newline='')

#Arrays to temporarily hold files and track lists
importedJSONArray = []
processedTracks = []

#Create pattern and file list from pattern
jsonPattern = os.path.join(pathToJson,'*.json')
fileList = glob.glob(jsonPattern)

#Loop through each file in the file list, open it, and append the data to the 
for file in fileList:
  with open(file) as f:
    json_data = pd.json_normalize(json.loads(f.read()))
    importedJSONArray.append(json_data)

#Combine the json files into a Pandas Data Frame
jsonCombinedDf = pd.concat(importedJSONArray, ignore_index=True)

#Get the number total number of tracks in the dataframe by getting its length and subtracting 1 (index starts at 0)
#You could change this if you only want to process 100 tracks or something
numTracks = (len(jsonCombinedDf)-1)

#Loop to extract relevant data from the jsonCombinedDf
for x in range(numTracks):
  #Only select tracks that were played the minimum amount of time. 
  #Spotify records that time in ms, so we multiply the minimum number of seconds by 1000
  if jsonCombinedDf.ms_played[x] >= (minPlayTime*1000):
    #Get the artist, album, track, and date/time played
    trackArtist = jsonCombinedDf.master_metadata_album_artist_name[x]
    trackAlbum = jsonCombinedDf.master_metadata_album_album_name[x]
    trackName = jsonCombinedDf.master_metadata_track_name[x]
    dateTimePlayed = jsonCombinedDf.ts[x]

    #Extract the relevant parts from the dateTimePlayedString using substrings
    day = dateTimePlayed[8:10]
    monthNum = dateTimePlayed[5:7]
    year = dateTimePlayed[0:4]
    time = dateTimePlayed[11:16]

    #Convert the month number into the short abbreviation of the month name
    monthConverter = datetime.datetime.strptime(monthNum, "%m")
    monthName = monthConverter.strftime("%b")
    #Construct a date/time string in our desired format
    convertedDateTimeString = day + ' ' + monthName + ' ' + year + ' ' + time

    #Construct a finished row by combining the artist, album, track name, and converted date time string
    finishedRow = (str(trackArtist)+','+str(trackAlbum)+','+str(trackName)+','+str(convertedDateTimeString))
    #Add the finished row to the processedTracks array that is temporarily holding our list of tracks
    processedTracks.append(finishedRow)

#Create a Pandas Data Frame from the processedTracks array
completeDF = pd.DataFrame(processedTracks)

#Create our final CSV file from the data frame we just created
completeDF.to_csv('output.csv', header = False, index = False, sep="\t", quoting=csv.QUOTE_NONE, quotechar="", escapechar="")

57403
